In [13]:
# %pip install sqlalchemy En caso de ser necesario, ejecutar
# %pip install --user psycopg2 En caso de ser necesario, ejecutar
# %pip install numpy En caso de ser necesario, ejecutar
# %pip install pandas En caso de ser necesario, ejecutar

In [14]:
import yaml
import pandas as pd
from sqlalchemy import create_engine

In [15]:
with open('config.yaml') as f:
    config = yaml.safe_load(f)
    configFuente = config['fuente']
    configBodega = config['bodega']

urlFuente = f'{configFuente['driver']}://{configFuente['user']}:{configFuente['password']}@{configFuente['host']}:{configFuente['port']}/{configFuente['db']}'
urlBodega = f'{configBodega['driver']}://{configBodega['user']}:{configBodega['password']}@{configBodega['host']}:{configBodega['port']}/{configBodega['db']}'

src = create_engine(urlFuente)
etl = create_engine(urlBodega)

In [ ]:
mensajero1 = pd.read_sql_table('clientes_mensajeroaquitoy', src)
mensajero2 = pd.read_sql_table('auth_user', src)
mensajero1.drop(columns=['fecha_entrada', 'fecha_salida', 'salario', 'ciudad_operacion_id', 'token_Firebase', 'url_foto', 'activo'], inplace=True)
mensajero2.drop(columns=['password', 'last_login', 'is_superuser', 'username', 'is_staff', 'is_active', 'date_joined'], inplace=True)
mensajero = mensajero1.merge(mensajero2, left_on='user_id', right_on='id', how='left')
mensajero.drop(columns=['user_id', 'id_y'], inplace=True)
mensajero.rename(columns={
  'id_x': 'key_dim_mensajero',
  'first_name': 'nombre',
  'last_name' : 'apellido'}, inplace=True)

# Mover la columna 'telefono' al final
cols = [col for col in mensajero.columns if col != 'telefono'] + ['telefono']
mensajero = mensajero[cols]


mensajero

,key_dim_mensajero,nombre,apellido,email,telefono
0,1,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,310-300000
1,42,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,310-300000
2,48,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,310-300000
3,41,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,310-300000
4,13,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,310-300000
5,28,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,310-300000
6,33,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,310-300000
7,36,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,310-300000
8,8,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,310-300000
9,21,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,310-300000


In [24]:
mensajero.to_sql('dim_mensajero', etl, index_label='key_dim_mensajero', index=False,  if_exists='replace')

50